In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 3.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [142]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_e233adc275fa4742979f13e05f7ec708.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [143]:
df=df.fillna(df.median())
df.shape

(961, 6)

In [144]:
df=df.rename(columns={0:"BI-RAD Assessment",1:"Age",2:"Shape",3:"Margin",4:"Density",5:"Severity"})
df.head()

,BI-RAD Assessment,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,3.0,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,3.0,1


In [145]:
df.groupby("Severity").count().T

Severity,0,1
BI-RAD Assessment,516,445
Age,516,445
Shape,516,445
Margin,516,445
Density,516,445


In [146]:
X=df.drop(["Severity"],axis=1)
X.head()

,BI-RAD Assessment,Age,Shape,Margin,Density
0,5.0,67.0,3.0,5.0,3.0
1,4.0,43.0,1.0,1.0,3.0
2,5.0,58.0,4.0,5.0,3.0
3,4.0,28.0,1.0,1.0,3.0
4,5.0,74.0,1.0,5.0,3.0


In [147]:
y=df[["Severity"]]
y.head()

,Severity
0,1
1,1
2,1
3,0
4,1


In [148]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

,BI-RAD Assessment,Age,Shape,Margin,Density
0,0.366477,0.796984,0.220384,1.436762,0.224804
1,-0.195221,-0.865610,-1.415052,-1.183216,0.224804
2,0.366477,0.173511,1.038102,1.436762,0.224804
3,-0.195221,-1.904732,-1.415052,-1.183216,0.224804
4,0.366477,1.281908,-1.415052,1.436762,0.224804


In [149]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(672, 5)
(289, 5)


In [150]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(n_estimators=190,criterion="entropy",max_depth=3,min_samples_leaf=2,max_features="sqrt",
                              bootstrap=False,ccp_alpha=0.01,n_jobs=-1,random_state=42).fit(X_train,y_train)
Forest

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=False, ccp_alpha=0.01, criterion='entropy',
                       max_depth=3, max_features='sqrt', min_samples_leaf=2,
                       n_estimators=190, n_jobs=-1, random_state=42)

In [151]:
from sklearn.metrics import accuracy_score
ytrain_pred=Forest.predict(X_train)
ytest_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8392857142857143
Accuracy on test data:  0.8408304498269896


In [152]:
from sklearn.metrics import roc_auc_score
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.8356634955102477
AUC on test data:  0.8384240494113107


In [153]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(loss="deviance",learning_rate=0.06,criterion="mse",max_depth=1,max_features="log2",
                                    min_samples_split=3,random_state=42).fit(X_train,y_train)
Gradient

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingClassifier(criterion='mse', learning_rate=0.06, max_depth=1,
                           max_features='log2', min_samples_split=3,
                           random_state=42)

In [154]:
ytrain_pred=Gradient.predict(X_train)
ytest_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8392857142857143
Accuracy on test data:  0.8373702422145328


In [155]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np

BForest=BalancedRandomForestClassifier(n_estimators=210,criterion="gini",max_depth=2,min_samples_split=3,
                                       min_samples_leaf=1,max_features="auto",bootstrap=True,
                                       sampling_strategy="not minority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(max_depth=2, min_samples_split=3,
                               n_estimators=210, n_jobs=-1, random_state=0,
                               sampling_strategy='not minority')

In [156]:
ytrain_pred=BForest.predict(X_train)
ytest_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8363095238095238
Accuracy on test data:  0.8442906574394463


In [157]:
from sklearn.metrics import roc_auc_score
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.837790454492867
AUC on test data:  0.8470372514958503


In [158]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
BBagging=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),n_estimators=70,max_features=2,
                                 bootstrap=True,bootstrap_features=True,sampling_strategy="majority",replacement=False,
                                 random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                          bootstrap_features=True, max_features=2,
                          n_estimators=70, random_state=0,
                          sampling_strategy='majority')

In [159]:
ytrain_pred=BBagging.predict(X_train)
ytest_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8407738095238095
Accuracy on test data:  0.8339100346020761


In [160]:
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.8427874731905275
AUC on test data:  0.836879945956379


In [163]:
X=df.drop(["Severity"],axis=1)
X.head()

,BI-RAD Assessment,Age,Shape,Margin,Density
0,5.0,67.0,3.0,5.0,3.0
1,4.0,43.0,1.0,1.0,3.0
2,5.0,58.0,4.0,5.0,3.0
3,4.0,28.0,1.0,1.0,3.0
4,5.0,74.0,1.0,5.0,3.0


In [164]:
import numpy as np
X_sub=X.drop(["Age"],axis=1)
X_sub=2*(np.sqrt(X_sub)-1)

In [165]:
X_1=df[["Age"]]
X_1=pd.DataFrame(scaler.fit_transform(X_1))

In [166]:
X=pd.concat([X_sub,X_1],axis=1)

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(672, 5)
(289, 5)


In [168]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(n_estimators=220,criterion="entropy",max_depth=3,min_samples_leaf=1,max_features="sqrt",
                              bootstrap=False,ccp_alpha=0.01,n_jobs=-1,random_state=42).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(bootstrap=False, ccp_alpha=0.01, criterion='entropy',
                       max_depth=3, max_features='sqrt', n_estimators=220,
                       n_jobs=-1, random_state=42)

In [169]:
from sklearn.metrics import accuracy_score
ytrain_pred=Forest.predict(X_train)
ytest_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.84375
Accuracy on test data:  0.8477508650519031


In [170]:
from sklearn.metrics import roc_auc_score
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.8427073785008055
AUC on test data:  0.847809303223316


In [171]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix on train data: \n",confusion_matrix(y_train,ytrain_pred))
print("Confusion matrix on test data: \n",confusion_matrix(y_test,ytest_pred))

Confusion matrix on train data: 
 [[308  51]
 [ 54 259]]
Confusion matrix on test data: 
 [[133  24]
 [ 20 112]]


In [172]:
print("On test data: \n")
print("Specificity: ",135/(135+22))
print("Sensitivity: ",109/(109+23))

On test data: 

Specificity:  0.8598726114649682
Sensitivity:  0.8257575757575758


In [173]:
y_pred=pd.DataFrame(Forest.predict_proba(X_test)).rename(columns={0:"pred_0",1:"pred_1"})
y_pred.head()

,pred_0,pred_1
0,0.861147,0.138853
1,0.158551,0.841449
2,0.158551,0.841449
3,0.926510,0.073490
4,0.565271,0.434729


In [174]:
y_probs1=y_pred[["pred_1"]]

In [175]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [176]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [177]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [178]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.489, 0.8512110726643599)


In [179]:
len(y_probs1[y_probs1["pred_1"]>=0.489])

143

In [180]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.489),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.489),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.489),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 <0.489),0,y_pred.pred_1)

In [181]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  143


In [182]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)

In [183]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  146


In [184]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
y_true=pd.DataFrame(enc.fit_transform(y_test).toarray())

In [185]:
y_pred=enc.inverse_transform(y_pred)

In [186]:
from sklearn.metrics import accuracy_score
print("Accuracy after adjusting the threshold: ",accuracy_score(y_test,y_pred))

Accuracy after adjusting the threshold:  0.8512110726643599


In [187]:
print("AUC after adjusting the threshold: ",roc_auc_score(y_test,y_pred))

AUC after adjusting the threshold:  0.8534066782474427


In [188]:
print("Confusion_matrix after adjusting the threshold: \n",confusion_matrix(y_test,y_pred))

Confusion_matrix after adjusting the threshold: 
 [[130  27]
 [ 16 116]]


In [189]:
print("On the test data: \n")
print("Specificity: ",130/(130+27))
print("Sensitivity: ",116/(116+16))

On the test data: 

Specificity:  0.8280254777070064
Sensitivity:  0.8787878787878788


In [197]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(n_estimators=85,criterion="entropy",max_depth=5,min_samples_split=3,
                                       min_samples_leaf=1,max_features="sqrt",bootstrap=True,
                                       sampling_strategy="not minority",replacement=True,n_jobs=-1,warm_start=False,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(criterion='entropy', max_depth=5,
                               max_features='sqrt', min_samples_split=3,
                               n_estimators=85, n_jobs=-1, random_state=0,
                               replacement=True,
                               sampling_strategy='not minority')

In [198]:
from sklearn.metrics import accuracy_score
ytrain_pred=BForest.predict(X_train)
ytest_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8556547619047619
Accuracy on test data:  0.8546712802768166


In [199]:
from sklearn.metrics import roc_auc_score
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.8552822447871706
AUC on test data:  0.854781895386991


In [200]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for test data: \n",confusion_matrix(y_test,ytest_pred))

Confusion matrix for test data: 
 [[134  23]
 [ 19 113]]


In [201]:
print("Specificity: ",134/(134+23))
print("Sensitivity: ",113/(113+19))

Specificity:  0.8535031847133758
Sensitivity:  0.8560606060606061


In [202]:
y_pred=pd.DataFrame(BForest.predict_proba(X_test)).rename(columns={0:"pred_0",1:"pred_1"})
y_pred.head()

,pred_0,pred_1
0,0.876175,0.123825
1,0.130932,0.869068
2,0.135185,0.864815
3,0.958705,0.041295
4,0.630789,0.369211


In [203]:
y_probs1=y_pred[["pred_1"]]

In [204]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [205]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [206]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [207]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.458, 0.8546712802768167)


In [208]:
len(y_probs1[y_probs1["pred_1"]>=0.458])

144

In [209]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.458),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.458),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.458),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 <0.458),0,y_pred.pred_1)

In [210]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  144


In [211]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)

In [212]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  145


In [213]:
y_pred=enc.inverse_transform(y_pred)

In [214]:
from sklearn.metrics import accuracy_score
print("Accuracy on test data after adjusting the threshold: ",accuracy_score(y_test,y_pred))

Accuracy on test data after adjusting the threshold:  0.8546712802768166


In [215]:
print("AUC on test data after adjusting the threshold: ",roc_auc_score(y_test,y_pred))

AUC on test data after adjusting the threshold:  0.8571945570353214


In [216]:
print("Confusion_matrix on test data after adjusting the threshold: \n",confusion_matrix(y_test,y_pred))

Confusion_matrix on test data after adjusting the threshold: 
 [[130  27]
 [ 15 117]]


In [217]:
print("After adjusting the threshold: \n")
print("Test data specificity: ",130/(130+27))
print("Test data sensitivity: ",117/(117+15))

After adjusting the threshold: 

Test data specificity:  0.8280254777070064
Test data sensitivity:  0.8863636363636364


In [145]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(loss="exponential",learning_rate=0.075,criterion="mse",max_depth=1,max_features="sqrt",
                                    min_samples_split=2,min_samples_leaf=1,random_state=42).fit(X_train,y_train)
Gradient

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingClassifier(criterion='mse', learning_rate=0.075,
                           loss='exponential', max_depth=1, max_features='sqrt',
                           random_state=42)

In [146]:
ytrain_pred=Gradient.predict(X_train)
ytest_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.8392857142857143
Accuracy on test data:  0.8442906574394463


In [147]:
print("AUC on train data: ",roc_auc_score(y_train,ytrain_pred))
print("AUC on test data: ",roc_auc_score(y_test,ytest_pred))

AUC on train data:  0.8366869276566964
AUC on test data:  0.842815093611272


### Conclusion:
The Random Forest model with data transformation and threshold adjustment is the most promising model.